Importing Data and packages

In [85]:
from importlib import reload
from pyvis.network import Network
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import params as params
import data.case30 as grid

reload(grid)

line_data = pd.DataFrame(grid.case30()['branch'],
                     columns=['fbus', 'tbus', 'r', 'x', 'b', 'rateA', 'rateB', 'rateC', 'ratio', 'angle', 'status', 'angmin', 'angmax'])
line_data = line_data.astype({'fbus': int, 'tbus': int, 'status': int})

bus_data = pd.DataFrame(grid.case30()['bus'],
                        columns=['bus_i', 'type', 'Pd', 'Qd', 'Gs', 'Bs', 'area', 'Vm', 'Va', 'baseKV', 'zone', 'Vmax', 'Vmin'])
bus_data = bus_data.astype({'bus_i': int, 'type': int})

gen_data = pd.DataFrame(grid.case30()['gen'], columns=['bus', 'Pg', 'Qg', 'Qmax', 'Qmin', 'Vg', 'mBase', 'status', 'Pmax', 'Pmin', 'Pc1', 'Pc2',
     'Qc1min', 'Qc1max', 'Qc2min', 'Qc2max', 'ramp_agc', 'ramp_10', 'ramp_30', 'ramp_q', 'apf'])
gen_data = gen_data.astype({'bus': int, 'status': int})

print(line_data)

DERs = set({1, 2, 22, 27, 23, 13})
CLs = set({2, 7, 8, 12, 21, 30})


    fbus  tbus     r     x     b  rateA  rateB  rateC  ratio  angle  status  \
0      1     2  0.02  0.06  0.03  130.0  130.0  130.0    0.0    0.0       1   
1      1     3  0.05  0.19  0.02  130.0  130.0  130.0    0.0    0.0       1   
2      2     4  0.06  0.17  0.02   65.0   65.0   65.0    0.0    0.0       1   
3      3     4  0.01  0.04  0.00  130.0  130.0  130.0    0.0    0.0       1   
4      2     5  0.05  0.20  0.02  130.0  130.0  130.0    0.0    0.0       1   
5      2     6  0.06  0.18  0.02   65.0   65.0   65.0    0.0    0.0       1   
6      4     6  0.01  0.04  0.00   90.0   90.0   90.0    0.0    0.0       1   
7      6     7  0.03  0.08  0.01  130.0  130.0  130.0    0.0    0.0       1   
8      6     8  0.01  0.04  0.00   32.0   32.0   32.0    0.0    0.0       1   
9      6     9  0.00  0.21  0.00   65.0   65.0   65.0    0.0    0.0       1   
10     6    10  0.00  0.56  0.00   32.0   32.0   32.0    0.0    0.0       1   
11     9    11  0.00  0.21  0.00   65.0   65.0   65.

Adding edges and weights to graph

In [86]:
G = nx.Graph()
wt = {}
for u, v, w in zip(line_data['fbus'], line_data['tbus'], line_data['status']):
    G.add_edge(u, v)
    wt[(u, v)] = w
    wt[(v, u)] = w

Finding all simple paths from each critical load to each source

In [87]:
path_list = []
for cl in CLs:
    tmp = []
    for path in nx.all_simple_paths(G, source=cl, target=DERs):
        loads, sources = 0, 0
        for node in path:
            if node in CLs:
                loads += 1
            if node in DERs:
                sources += 1
        if loads > 1 or sources > 1:
            continue
        tmp.append(list(nx.utils.pairwise(path)))
    if len(tmp) == 0: continue
    path_list.append(tmp)
print(path_list)


[[[(21, 10), (10, 6), (6, 4), (4, 3), (3, 1)], [(21, 10), (10, 6), (6, 28), (28, 27)], [(21, 10), (10, 9), (9, 6), (6, 4), (4, 3), (3, 1)], [(21, 10), (10, 9), (9, 6), (6, 28), (28, 27)], [(21, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(21, 10), (10, 22)], [(21, 22)]], [[(7, 6), (6, 4), (4, 3), (3, 1)], [(7, 6), (6, 9), (9, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(7, 6), (6, 9), (9, 10), (10, 22)], [(7, 6), (6, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(7, 6), (6, 10), (10, 22)], [(7, 6), (6, 28), (28, 27)]], [[(8, 6), (6, 4), (4, 3), (3, 1)], [(8, 6), (6, 9), (9, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(8, 6), (6, 9), (9, 10), (10, 22)], [(8, 6), (6, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(8, 6), (6, 10), (10, 22)], [(8, 6), (6, 28), (28, 27)], [(8, 28), (28, 27)], [(8, 28), (28, 6), (6, 4), (4, 3), (3, 1)], [(8, 28), (28, 6), (6, 9), (9, 10), (10, 20), (20, 19), (19, 18), (18, 15), (15, 23)], [(8, 28), 

All possible path combination calculation

In [88]:
import itertools

FNs = []
raw_combs = []
for element in itertools.product(*path_list):
    tmp = []
    for path in element:
        for edge in path:
            tmp.append(edge)
    raw_combs.append(list(element))
    FNs.append(tmp)


In [89]:
for node in G.nodes:
    G.nodes[node]['label'] = f"{node}"
    if node in CLs:
        G.nodes[node]['color'] = '#ff0000'
    elif node in DERs:
        G.nodes[node]['color'] = '#00ff00'
    else:
        G.nodes[node]['color'] = '#ffffff'

graph = Network(notebook=True, cdn_resources='local')

options = {
  "physics": {
    "forceAtlas2Based": {
      "theta": 0.2,
      "gravitationalConstant": -41,
      "springLength": 40,
      "springConstant": 0.53
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}
graph.from_nx(G)
graph.options = options
graph.save_graph('graph.html')

### Calculating parameters to measure resiliency of possible network

Calculation of Parameter matrix

In [90]:
rsl = params.ratio_source_load(FNs, CLs, DERs)
ops = params.switch_ops(FNs, wt)
cen = params.agg_centrality(FNs)
obs = params.overlapping_branches(FNs)
apl = params.avg_path_length(raw_combs)

df = pd.DataFrame(np.array([rsl, ops, cen, obs, apl]), columns=[f"FN{i}" for i in range(1, len(rsl)+1)])
df.index = ['rsl', 'ops', 'cen', 'obs', 'apl']
df = df.T
print(df)

              rsl  ops       cen       obs  apl
FN1      2.500000  0.0  0.163636  0.470588  3.4
FN2      2.500000  0.0  0.151515  0.444444  3.6
FN3      1.666667  0.0  0.117647  0.260870  4.6
FN4      1.666667  0.0  0.111111  0.250000  4.8
FN5      1.666667  0.0  0.153846  0.315789  3.8
...           ...  ...       ...       ...  ...
FN25196  2.500000  0.0  0.153846  0.117647  3.4
FN25197  1.666667  0.0  0.117647  0.000000  3.4
FN25198  1.666667  0.0  0.111111  0.000000  3.6
FN25199  2.500000  0.0  0.166667  0.076923  2.6
FN25200  2.500000  0.0  0.153846  0.071429  2.8

[25200 rows x 5 columns]


Calculation of weighted mean and resiliency

In [91]:
df['wtd_mean'] = (df['rsl'] * 6 + df['ops'] * 3 + df['cen'] * 1 + df['obs'] * 5 + df['apl'] * 5)/20
df['resiliency'] = np.exp(-df['wtd_mean'])
print(df.sort_values('resiliency', ascending=False))

          rsl  ops       cen       obs  apl  wtd_mean  resiliency
FN21913  1.25  0.0  0.115385  0.000000  1.8  0.830769    0.435714
FN21937  1.25  0.0  0.109890  0.000000  2.0  0.880495    0.414578
FN18313  1.25  0.0  0.109890  0.000000  2.0  0.880495    0.414578
FN21914  1.25  0.0  0.109890  0.000000  2.0  0.880495    0.414578
FN21863  1.25  0.0  0.128205  0.000000  2.0  0.881410    0.414198
...       ...  ...       ...       ...  ...       ...         ...
FN6872   5.00  0.0  0.133333  0.272727  4.4  2.674848    0.068917
FN14122  5.00  0.0  0.125000  0.318182  4.4  2.685795    0.068167
FN14084  5.00  0.0  0.133333  0.363636  4.4  2.697576    0.067369
FN14071  5.00  0.0  0.133333  0.363636  4.4  2.697576    0.067369
FN14072  5.00  0.0  0.125000  0.347826  4.6  2.743207    0.064364

[25200 rows x 7 columns]


Load flow analysis of the networks

In [92]:
%%bash
pf data/case30.py

PYPOWER Version 5.1.16, 05-March-2023 -- AC Power Flow (Newton)


Newton's method power flow converged in 3 iterations.

Converged in 0.01 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             30     Total Gen Capacity     335.0         -95.0 to 405.9
Generators         6     On-line Capacity       335.0         -95.0 to 405.9
Committed Gens     6     Generation (actual)    192.1             102.8
Loads             20     Load                   189.2             107.2
  Fixed           20       Fixed                189.2             107.2
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             2     Shunt (inj)             -0.0               0.2
Branches          40     Losses (I^2 * Z)         2.86             10.37
Transformers       0     Branc